In [1]:
from __future__ import absolute_import, division, print_function
import json
from collections import Counter
import os
import argparse
from math import log
from tqdm.auto import tqdm
from easydict import EasyDict as edict
import torch
from functools import reduce
from kg_env_m import BatchKGEnvironment
from actor_critic import ActorCritic
from utils import *
import wandb

/Users/guntsv/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2.0 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [2]:
class Arguemnt:
    seed = 0
    config = "config/beauty/graph_reasoning/UPGPR.json"
    
args = Arguemnt()

with open(args.config, "r") as f:
    config = edict(json.load(f))

config.seed = args.seed
config_agent = config.AGENT

if config.use_wandb:
    wandb.init(
        project=config.wandb_project_name,
        name=config.wandb_run_name,
        config=config,
    )

os.environ["CUDA_VISIBLE_DEVICES"] = config_agent.gpu
config_agent.device = torch.device("cuda:0") if torch.cuda.is_available() else "cpu"

if config_agent.early_stopping == True:
    with open("early_stopping.txt", "r") as f:
        config_agent.epochs = int(f.read())

config_agent.log_dir = config.processed_data_dir + "/" + config_agent.name
# test(config)

# if config.use_wandb:
#     wandb.finish()

In [4]:
config_agent = config.AGENT
kg_config = config.KG_ARGS

policy_file = config_agent.log_dir + "/tmp_policy_model_epoch_{}.ckpt".format(
    config_agent.epochs
)
path_file = config_agent.log_dir + "/policy_paths_epoch_{}.pkl".format(
    config_agent.epochs
)

train_labels = load_labels(config.processed_data_dir, "train")
test_labels = load_labels(config.processed_data_dir, "test")

dataset_name = config.processed_data_dir.split("/")[-1]

model_name = (
    "UPGPR_len_"
    + str(config_agent.max_path_len)
    + "_"
    + config.AGENT.reward
    + "_"
    + config.TRAIN_EMBEDS.cold_start_embeddings
    + "_mask_"
    + str(config.AGENT.mask_first_interaction)
    + "_max_cold_concept_"
    + str(kg_config.max_nb_cold_entities)
    + "_topk_"
    + "_".join(map(str, config_agent.topk))
)

config_agent.result_file_dir = os.path.join(
    config_agent.result_file_dir, dataset_name, model_name, str(config.seed)
)

os.makedirs(
    config_agent.result_file_dir,
    exist_ok=True,
)

In [5]:
# if config_agent.run_path:
#     predict_paths(
#         policy_file, 
#         path_file, config, 
#         config_agent, 
#         kg_config
#     )

In [7]:
##def predict_paths
from test_agent import batch_beam_search, evaluate

set_name = 'test'
print("Predicting paths...")
env = BatchKGEnvironment(
    config.processed_data_dir,
    kg_config,
    set_name=set_name,
    max_acts=config_agent.max_acts,
    max_path_len=config_agent.max_path_len,
    state_history=config_agent.state_history,
    reward_function=config_agent.reward,
    mask_first_interaction=True,
    use_pattern=config_agent.use_pattern,
)
pretrain_sd = torch.load(policy_file, map_location=torch.device("cpu"))
model = ActorCritic(
    env.state_dim,
    env.act_dim,
    gamma=config_agent.gamma,
    hidden_sizes=config_agent.hidden,
    modified_policy=config_agent.modified_policy,
    embed_size=env.embed_size,
).to(config_agent.device)
model_sd = model.state_dict()
model_sd.update(pretrain_sd)
model.load_state_dict(model_sd)

test_labels = load_labels(config.processed_data_dir, set_name)
test_uids = list(test_labels.keys())[:201]

batch_size = 16
start_idx = 0
all_paths, all_probs = [], []
pbar = tqdm(total=len(test_uids))
while start_idx < len(test_uids):
    end_idx = min(start_idx + batch_size, len(test_uids))
    batch_uids = test_uids[start_idx:end_idx]
    paths, probs = batch_beam_search(
        env,
        model,
        kg_config,
        batch_uids,
        config_agent.device,
        topk=config_agent.topk,
        policy=config_agent.modified_policy,
    )
    all_paths.extend(paths)
    all_probs.extend(probs)
    start_idx = end_idx
    pbar.update(batch_size)
predicts = {"paths": all_paths, "probs": all_probs}
pickle.dump(predicts, open(path_file, "wb"))
if config.use_wandb:
    wandb.save(path_file)

Predicting paths...
Orginal kg.G.keys dict_keys(['user', 'item', 'brand', 'category', 'related_product', 'word', 'feature'])
delete kg.G['feaure']
Updated kg.G.keys dict_keys(['user', 'item', 'brand', 'category', 'related_product', 'word'])
Load embedding: data/beauty/Amazon_Beauty_01_01/train_transe_embed.pkl
Key embed dict_keys(['user', 'item', 'brand', 'category', 'related_product', 'word', 'also_bought', 'also_viewed', 'bought_together', 'described', 'belong_to', 'category_of', 'mentioned', 'interested_in', 'like', 'dislike', 'purchase'])


  0%|          | 0/201 [00:00<?, ?it/s]

In [12]:
# predicts["paths"]

In [ ]:
# if config_agent.run_eval:
#     evaluate_paths(
#         config.processed_data_dir,
#         path_file,
#         train_labels,
#         test_labels,
#         kg_config,
#         config.use_wandb,
#         config_agent.result_file_dir,
#         validation=False,
#     )

In [13]:
##def evaluate_paths(
dir_path=config.processed_data_dir
embeds = load_embed(dir_path, set_name)
# embeds = load_embed(dir_path, 'train')
user_embeds = embeds["user"]
interaction_embeds = embeds[kg_config.interaction][0]
item_embeds = embeds["item"]
scores = np.dot(user_embeds + interaction_embeds, item_embeds.T)

Load embedding: data/beauty/Amazon_Beauty_01_01/test_transe_embed.pkl


In [14]:
# 1) Get all valid paths for each user, compute path score and path probability.
results = pickle.load(open(path_file, "rb"))
pred_paths = {uid: {} for uid in test_labels}
for path, probs in zip(results["paths"], results["probs"]):
    if path[-1][1] != "item":
        continue
    uid = path[0][2]
    if uid not in pred_paths:
        continue
    pid = path[-1][2]
    if pid not in pred_paths[uid]:
        pred_paths[uid][pid] = []
    path_score = scores[uid][pid]
    path_prob = reduce(lambda x, y: x * y, probs)
    pred_paths[uid][pid].append((path_score, path_prob, path))

In [21]:
# 3) Pick best path for each user-product pair, also remove pid if it is in train set.
best_pred_paths = {}
for uid in pred_paths:
    train_pids = set(train_labels.get(uid, []))
    # if len(train_pids) == 0:
    #     continue
    best_pred_paths[uid] = []
    for pid in pred_paths[uid]:
        if pid in train_pids:
            continue
        # Get the path with highest probability
        sorted_path = sorted(pred_paths[uid][pid], key=lambda x: x[1], reverse=True)
        best_pred_paths[uid].append(sorted_path[0])

In [22]:
# 3) Compute top 10 recommended products for each user.
sort_by = "score"
pred_labels = {}
for uid in best_pred_paths:
    if sort_by == "score":
        sorted_path = sorted(
            best_pred_paths[uid], key=lambda x: (x[0], x[1]), reverse=True
        )
    elif sort_by == "prob":
        sorted_path = sorted(
            best_pred_paths[uid], key=lambda x: (x[1], x[0]), reverse=True
        )
    top_pids = [p[-1][2] for _, _, p in sorted_path]  # from largest to smallest

    pred_labels[uid] = top_pids[:10]  # change order to from smallest to largest!

In [23]:
pred_labels

{5164: [43, 874, 617, 725, 9, 167, 235, 2458, 2675],
 1333: [43, 734, 72, 106, 247, 195, 37, 687, 167, 91],
 12080: [1155, 463, 278, 324, 68, 936, 84, 151, 355],
 867: [161, 839, 1506, 1050, 1122, 43, 608, 840],
 12887: [821, 838, 15, 1114, 5, 76, 92, 219],
 3563: [186, 161, 68, 676, 968, 15, 195],
 297: [1389, 1156, 1030, 63, 1396, 133],
 22289: [1506, 43, 725, 1731, 839, 161, 823, 566, 937, 85],
 6214: [51, 473, 56, 1236, 617, 217, 1496, 44, 314, 161],
 21576: [4926, 161, 806, 367, 76, 43, 134],
 378: [76, 499, 5630, 23, 161, 134, 676, 43],
 20047: [43, 806, 901, 137, 35, 734, 1578, 82, 2026],
 3167: [43, 62, 676, 111, 1426, 65, 1110, 1802, 6452, 6782],
 6045: [106, 415, 1251, 167, 43, 286, 1174, 3263, 3935],
 15355: [1453, 1062, 676, 593, 161, 43, 5486, 4329, 11871],
 4492: [695, 439, 38, 15, 562, 65, 2690, 3, 826],
 12566: [2673, 23, 124, 436, 132, 1313, 246, 320, 245],
 12757: [43, 734, 581, 1076, 921, 23, 161, 36, 120],
 21991: [303, 445, 6251, 161, 167, 4865, 43, 676],
 3012: [5

In [24]:
test_labels

{5164: [2085],
 1333: [10758],
 12080: [9151],
 867: [1643, 1811],
 12887: [4919, 7614, 9578, 7623],
 3563: [9918, 2171],
 297: [6846],
 22289: [9576],
 6214: [10865],
 21576: [1284],
 378: [5630],
 20047: [3541],
 3167: [11369],
 6045: [7186],
 15355: [5486],
 4492: [10371],
 12566: [9063, 8923, 2171, 7309, 9706, 167, 8180],
 12757: [1057],
 21991: [6856],
 3012: [2339],
 5373: [4870],
 4248: [2246],
 1259: [4158],
 4598: [6424],
 7562: [503],
 14357: [7182],
 1749: [11153],
 2189: [2910],
 16766: [5845],
 18384: [12005],
 1005: [11586],
 20135: [2325],
 9841: [8407],
 12552: [10907],
 7344: [9617],
 1063: [11133],
 3645: [5603],
 10969: [3200],
 4887: [3637],
 3606: [6299],
 9079: [10772],
 20406: [1901],
 13371: [10640],
 21617: [6021],
 11703: [1430],
 17322: [9523],
 13036: [5112],
 10612: [8486],
 14620: [8374],
 22019: [4807],
 9033: [4165],
 15205: [9614],
 6756: [3266],
 18137: [7689],
 6400: [1975, 10969],
 12153: [6378],
 3648: [11632],
 18831: [5467],
 1187: [6408],
 12497:

In [27]:
use_wandb = False
result_file_dir = config_agent.result_file_dir
use_wandb = False

evaluate(
    pred_labels,
    test_labels,
    train_labels,
    use_wandb,
    config.processed_data_dir,
    result_file_dir=result_file_dir,
    min_items=10,
    compute_all=True,
)

NDCG=0.028 |  Recall=0.066 | HR=0.066 | Precision=0.007 | HR@1=0.005 | HR@3=0.016 | HR@5=0.022 | Computed for all users.



(0.0, 0.0, 0.0, 0.0)